In [4]:
from bs4 import BeautifulSoup
import re
import json
def clean_medical_text(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    raw_text = soup.get_text(separator='\n', strip=True)

    keep_chars = r'''
        a-zA-Z0-9    
        \u4e00-\u9fa5 
        \u3002\uFF0C  
        \uFF1A\uFF1B  
        \uFF08\uFF09  
        \u201C\u201D  
        \.$$\,\:\;  
        \s            
    '''
    # 先处理不可见字符
    cleaned_text = re.sub(r'[\x00-\x1F\x7F-\x9F\r\n]', '', raw_text)
    #删除所有的换行符
    cleaned_text = re.sub(r'\\n', '', cleaned_text)
    #删除所有的空格
    cleaned_text = re.sub(r' ', '', cleaned_text)
    #删除所有的制表符
    cleaned_text = re.sub(r'\\t', '', cleaned_text)
    #删除所有的回车符
    cleaned_text = re.sub(r'\\r', '', cleaned_text)

    return cleaned_text.strip()
import re

def extract_info(text):
    if re.search(r"入院记录", text):
        field_config = [
            ("档案类型", r"郑州大学第二附属医院(.*?)姓名"),
            ("姓名", r"姓名：([\u4e00-\u9fa5]+)科室"),
            ("性别", r"性别：([男女])"),
            ("年龄", r"年龄：(\d+)岁"),
            ("入院时间", r"入院时间：(.*?)职业"),
            ("记录时间", r"记录时间：(.*?)籍贯"),
            #("职业", r"职业：([\u4e00-\u9fa5（）\d]+)民族"),
            #("民族", r"民族：([\u4e00-\u9fa5]+)婚姻"),
            ("婚姻状况", r"婚姻：([\u4e00-\u9fa5]+)记录时间"),
            ("可靠程度", r"可靠程度：(.*?)工作单位"),
            #("籍贯", r"籍贯:([\u4e00-\u9fa5]+)入院情况"),
            #("联系方式", r"联系方式：([\d-]+)"),
            #("现住址", r"现住址:([\u4e00-\u9fa5\s\d]+)陈述者"),
            ("主诉", r"主诉:(.*?)。"),
            ("现病史", r"现病史:(.*?)既往史"),
            ("既往史", r"既往史:(.*?)个人史"),
            ("个人史", r"个人史:(.*?)婚育史："),
            ("婚育史", r"婚育史：(.*?)家族史:"),
            ("家族史", r"家族史:(.*?)患者信息及病史采集真实准确"),
            ("初步诊断", r"初步诊断(.*?)住院医师"),
            ("住院医师", r"住院医师：(.*?)主治医师"),
            ("主治医师", r"主治医师：(.*?)主治医师签名"),
            ("主任医师", r"主任医师：(.*?)上级医师签名"),
            ("体格检查", r"体格检查(.*?)初步诊断")
        ]
    # 字段配置表（出院信息）
    elif re.search(r"出院记录", text):
        field_config = [
            ("档案类型", r"郑州大学第二附属医院(.*?)姓名"),
            ("姓名", r"姓名：([\u4e00-\u9fa5]+)科室"),
            ("性别", r"性别：([男女])"),
            ("年龄", r"年龄：(\d+)岁"),
            ("入院时间", r"入院日期:(.*?)出院日期"),
            ("出院日期", r"出院日期:(.*?)入院情况"),
            ("入院情况", r"入院情况:(.*?)入院诊断"),
            ("入院诊断", r"入院诊断:(.*?)诊疗经过"),
            ("诊疗经过", r"诊疗经过:(.*?)出院诊断"),
            ("出院诊断", r"出院诊断:(.*?)出院情况"),
            ("出院情况", r"出院情况:(.*?)出院医嘱"),
            ("出院医嘱", r"出院医嘱:(.*?)$")
        ]
    elif re.search(r"住院期间医患沟通记录", text):
        field_config = [
            ("档案类型", r"郑州大学第二附属医院(.*?)姓名"),
            ("姓名", r"姓名：([\u4e00-\u9fa5]+)科室"),
            ("性别", r"性别:([男女])"),
            ("年龄", r"年龄：(\d+)岁"),
            ("科室", r"科室名称：(.*?)床号"),
            ("床号", r"床号：(.*?)住院号"),
            ("住院号", r"住院号：(.*?)姓名"),
            ("沟通时间", r"沟通时间:(.*?)沟通地点"),
            ("沟通地点", r"沟通地点:(.*?)目前诊断"),
            ("目前诊断", r"目前诊断：(.*?)沟通内容"),
            ("沟通内容", r"沟通内容：?\s*(.*?)\s*医师签名"),
            ("与患者关系", r"与患者关系：(.*?)第")
        ]
    elif re.search(r"出院证", text):
        field_config = [
            ("档案类型", r"郑州大学第二附属医院(.*?)住院号"),
            ("住院号", r"住院号：(.*?)医疗保险证号"),
            ("姓名", r"姓名：([\u4e00-\u9fa5]+)性别"),
            ("性别", r"性别：([男女])"),
            ("年龄", r"年龄：(.*?)岁"),
            ("入院日期", r"入院日期：(.*?)出院日期"),
            ("出院日期", r"出院日期：(.*?)出院诊断"),
            ("出院诊断", r"出院诊断：(.*?)出院医嘱"),
            ("出院医嘱", r"出院医嘱:(.*?)(?=主任医师|副主任医师|住院医师)")
        ]
    elif re.search(r"病程记录", text):
        field_config = [
            ("档案类型", r"郑州大学第二附属医院(.*?)姓名"),
            ("姓名", r"姓名：([\u4e00-\u9fa5]+)科室"),
            ("科室", r"科室：(.*?)床号"),
            ("床号", r"床号：(.*?)住院号"),
            ("住院号", r"住院号：(\d{8})"),
            ("记录时间", r"([0-9]{4}-[0-9]{2}-[0-9]{4}:[0-9]{2})"),
            ("目前情况及建议", r"[0-9]{4}-[0-9]{2}-[0-9]{4}:[0-9]{2}(.*?)$")
        ]
    else:
        field_config = [
            ("档案类型", r"郑州大学第二附属医院(.*?)(?=姓名|门诊号|住院号|科室|会诊地点|尊敬的)"),
            ("姓名", r"姓名：([\u4e00-\u9fa5]+)科室"),
            ("科室", r"科室：(.*?)床号"),
            ("床号", r"床号：(.*?)住院号"),
            ("住院号", r"住院号：(\d{8})"),
            ("记录时间", r"([0-9]{4}-[0-9]{2}-[0-9]{4}:[0-9]{2})"),
            ("目前情况及建议", r"[0-9]{4}-[0-9]{2}-[0-9]{4}:[0-9]{2}(.*?)$")
        ]
    info_dict = {}

    for field, pattern in field_config:
        try:
            if isinstance(pattern, tuple):  # 处理段落型字段
                start_tag, end_tag = pattern
                regex = re.compile(f"{re.escape(start_tag)}(.*?){re.escape(end_tag)}", re.DOTALL)
                match = regex.search(text)
                info_dict[field] = match.group(1).strip() if match else ""
            else:  # 处理键值对型字段
                match = re.search(pattern, text)
                info_dict[field] = match.group(1).strip() if match else ""
        except Exception as e:
            info_dict[field] = ""
            print(f"Error extracting {field}: {str(e)}")

    # 对体格检查部分进行分级处理
    if '体格检查' in info_dict:
        physical_examination = info_dict['体格检查']
        sub_sections = {
            '基本生命体征': r'T:(.*?)皮肤：',
            '皮肤检查': r'皮肤：(.*?)淋巴结：',
            '淋巴结检查': r'淋巴结：(.*?)头部：',
            '头部检查': r'头部：(.*?)颈部：',
            '颈部检查': r'颈部：(.*?)胸部：',
            '胸部检查': r'胸部：(.*?)腹部：',
            '腹部检查': r'腹部：(.*?)肛门、直肠:',
            '肛门直肠检查': r'肛门、直肠:(.*?)外生殖器：',
            '外生殖器检查': r'外生殖器：(.*?)脊柱:',
            '脊柱检查': r'脊柱:(.*?)四肢：',
            '四肢检查': r'四肢：(.*?)神经系统检查',
            '神经系统检查': r'神经系统检查(.*?)辅助检查',
            '辅助检查': r'辅助检查(.*?)$'
        }
        graded_info = {}
        for section_name, section_pattern in sub_sections.items():
            match = re.search(section_pattern, physical_examination, re.DOTALL)
            if match:
                graded_info[section_name] = match.group(1).strip()
        info_dict['体格检查'] = graded_info

    return info_dict
def process_file(file_path):
    all_results = []
    line_count = 0
    try:
        full_path = f'{file_path}'
        with open(full_path, 'r', encoding='utf-8') as file:
            for line in file:
                if line.strip() == '':
                    continue
                if line_count >= 100:
                    break
                line = clean_medical_text(line)
                result = extract_info(line)
                all_results.append(result)
                line_count = line_count + 1
    except FileNotFoundError:
        print(f"错误：未找到文件 {full_path}。")
    except Exception as e:
        print(f"发生未知错误：{e}")
    return all_results


file_path = 'output_1.txt' 
results = process_file(file_path)

if results:
    json_result = json.dumps(results, ensure_ascii=False, indent=4)
    # 构建保存json文件的相对路径，保存到code文件夹下
    save_path = f'hospital_records.json' 
    with open(save_path, 'w', encoding='utf-8') as file:
        file.write(json_result)
    print("处理完成，结果已保存到 hospital_records.json。")

处理完成，结果已保存到 hospital_records.json。


In [5]:
text = '{"0073561501420191107173940564":"<!DOCTYPE HTML PUBLIC \"-//W3C//DTD HTML 4.0 Transitional//EN\">\r\n<html>\r\n<head>\r\n\t<meta http-equiv=\"content-type\" content=\"text/html; charset=gb2312\"/>\r\n\t<title></title>\r\n\t<meta name=\"generator\" content=\"LibreOffice 5.3.5.2 (Windows)\"/>\r\n\t<meta name=\"created\" content=\"2016-01-26T09:30:16.270000000\"/>\r\n\t<meta name=\"changedby\" content=\"南安超\"/>\r\n\t<meta name=\"changed\" content=\"2019-12-02T20:20:31.720000000\"/>\r\n\t<meta name=\"Info 1\" content=\"\"/>\r\n\t<meta name=\"Info 2\" content=\"\"/>\r\n\t<meta name=\"Info 3\" content=\"\"/>\r\n\t<meta name=\"Info 4\" content=\"\"/>\r\n\t<meta name=\"Protect\" content=\"FALSE\"/>\r\n\t<style type=\"text/css\">\r\n\t\t@page { size: 21cm 29.7cm; margin-left: 2.5cm; margin-right: 2.5cm; margin-top: 4.5cm; margin-bottom: 2cm }\r\n\t\tp { margin-bottom: 0.21cm }\r\n\t\ttd p { margin-bottom: 0cm }\r\n\t</style>\r\n</head>\r\n<body lang=\"zh-CN\" dir=\"ltr\">\r\n<div title=\"header\">\r\n\t<p align=\"center\" style=\"margin-bottom: 0cm; line-height: 150%\">\r\n\t<font face=\"宋体\"><font size=\"4\" style=\"font-size: 16pt\"><b>郑州大学第二附属医院</b></font></font></p>\r\n\t<p align=\"center\" style=\"margin-bottom: 0cm; line-height: 150%\"><font face=\"宋体\"><font size=\"4\" style=\"font-size: 16pt\"><b>病\r\n\t程 记 录</b></font></font></p>\r\n\t<table width=\"100%\" cellpadding=\"4\" cellspacing=\"0\">\r\n\t\t<col width=\"57*\">\r\n\t\t<col width=\"85*\">\r\n\t\t<col width=\"55*\">\r\n\t\t<col width=\"59*\">\r\n\t\t<tr valign=\"top\">\r\n\t\t\t<td width=\"22%\" style=\"border: none; padding: 0cm\">\r\n\t\t\t\t<p align=\"left\"><font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\">姓名：杜银兰</font></font></p>\r\n\t\t\t</td>\r\n\t\t\t<td width=\"33%\" style=\"border: none; padding: 0cm\">\r\n\t\t\t\t<p align=\"left\"><font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\">科室：北眼科病房二</font></font></p>\r\n\t\t\t</td>\r\n\t\t\t<td width=\"22%\" style=\"border: none; padding: 0cm\">\r\n\t\t\t\t<p align=\"left\"><font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\">床号：</font></font><font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\"><span lang=\"en-US\">70527</span></font></font></p>\r\n\t\t\t</td>\r\n\t\t\t<td width=\"23%\" style=\"border: none; padding: 0cm\">\r\n\t\t\t\t<p align=\"left\"><font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\">住院号：</font></font><font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\"><span lang=\"en-US\">00735615</span></font></font></p>\r\n\t\t\t</td>\r\n\t\t</tr>\r\n\t</table>\r\n\t<p style=\"margin-bottom: 1.25cm; border-top: none; border-bottom: 1px solid #000000; border-left: none; border-right: none; padding: 0cm\">\r\n\t<br/>\n\r\n\t</p>\r\n</div>\r\n<p align=\"left\" style=\"margin-bottom: 0cm; line-height: 150%\"><font face=\"Times New Roman, serif\"><span lang=\"en-US\"><font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\"><span style=\"font-weight: normal\">2019-11-01\r\n08:39</span></font></font><font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\"><b>\r\n    </b></font></font></span></font>\r\n</p>\r\n<p align=\"justify\" style=\"margin-bottom: 0cm; line-height: 150%\">\r\n<font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\">今日查房：患者神志清，精神可，睡眠可，饮食可，大小便正常，查体：右眼：视力</font></font><font face=\"Times New Roman, serif\"><span lang=\"en-US\"><font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\">0.1</font></font></span></font><font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\">，眼睑上睑水肿，无内外翻，无倒睫；泪点开放，泪囊区无红肿、无压痛，泪腺无肿大；巩膜无黄染、无结节，球结膜稍充血水肿；角膜透明，直径约</font></font><font face=\"Times New Roman, serif\"><span lang=\"en-US\"><font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\">11mm</font></font></span></font><font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\">，</font></font><font face=\"Times New Roman, serif\"><span lang=\"en-US\"><font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\">kp(-)</font></font></span></font><font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\">；前房中深，闪辉</font></font><font face=\"Times New Roman, serif\"><span lang=\"en-US\"><font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\">(-)</font></font></span></font><font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\">；虹膜纹理清晰，无萎缩，色泽正常，无新生血管；瞳孔直径约</font></font><font face=\"Times New Roman, serif\"><span lang=\"en-US\"><font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\">3mm</font></font></span></font><font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\">，圆形，对光反应灵敏；晶状体混浊；玻璃体腔硅油填充\r\n；眼底模糊见视盘边界清，色淡红，视网膜平复，激光斑清晰，眼压</font></font><font face=\"Times New Roman, serif\"><span lang=\"en-US\"><font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\">15.0mmHg</font></font></span></font><font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\">，。</font></font><font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\"><span style=\"font-weight: normal\">南安超主治医师查房后指示</span></font></font><font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\">：\r\n按时用药，定期复查。上级医师指示已执行。</font></font></p>\r\n<p align=\"right\" style=\"margin-bottom: 0cm; line-height: 200%\">\r\n<font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\">主治医师</font></font><font face=\"Times New Roman, serif\"><span lang=\"en-US\"><font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\">:<img src=\"0073561501420191107173940564_html_61ab05fe792f8b05.png\" name=\"2017112616091611982\" align=\"absmiddle\" vspace=\"2\" width=\"62\" height=\"30\" border=\"0\"/>\n</font></font></span></font><font color=\"#bdbdbd\"><font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\"><span style=\"display: none\">上级医师签名</span></font></font></font><font face=\"Times New Roman, serif\"><span lang=\"en-US\"><font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\">/\r\n</font></font></span></font><font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\">代住院医师</font></font><font face=\"Times New Roman, serif\"><span lang=\"en-US\"><font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\">:<img src=\"0073561501420191107173940564_html_61ab05fe792f8b05.png\" name=\"2017112616095411982\" align=\"absmiddle\" vspace=\"2\" width=\"62\" height=\"30\" border=\"0\"/>\n</font></font></span></font><font color=\"#bdbdbd\"><font face=\"宋体\"><font size=\"3\" style=\"font-size: 12pt\"><span style=\"display: none\">医师签名</span></font></font></font></p>\r\n<div title=\"footer\">\r\n\t<p align=\"center\" style=\"margin-top: 1cm; margin-bottom: 0cm; border-top: 1px solid #000000; border-bottom: none; border-left: none; border-right: none; padding: 0cm\">\r\n\t第 <sdfield type=PAGE subtype=RANDOM format=PAGE><font face=\"Times New Roman, serif\">1</font></sdfield>\r\n\t页</p>\r\n\t<p align=\"center\" style=\"margin-bottom: 0cm; border-top: 1px solid #000000; border-bottom: none; border-left: none; border-right: none; padding: 0cm\">\r\n\t<br/>\n\r\n\t</p>\r\n</div>\r\n</body>\r\n</html>"}'

In [6]:
text = clean_medical_text(text)

In [7]:
text

'{"0073561501420191107173940564":"郑州大学第二附属医院病程记录姓名：杜银兰科室：北眼科病房二床号：70527住院号：007356152019-11-0108:39今日查房：患者神志清，精神可，睡眠可，饮食可，大小便正常，查体：右眼：视力0.1，眼睑上睑水肿，无内外翻，无倒睫；泪点开放，泪囊区无红肿、无压痛，泪腺无肿大；巩膜无黄染、无结节，球结膜稍充血水肿；角膜透明，直径约11mm，kp(-)；前房中深，闪辉(-)；虹膜纹理清晰，无萎缩，色泽正常，无新生血管；瞳孔直径约3mm，圆形，对光反应灵敏；晶状体混浊；玻璃体腔硅油填充；眼底模糊见视盘边界清，色淡红，视网膜平复，激光斑清晰，眼压15.0mmHg，。南安超主治医师查房后指示：按时用药，定期复查。上级医师指示已执行。主治医师:上级医师签名/代住院医师:医师签名第1页"}'

In [8]:
archive_types = set()

# 遍历列表中的每个字典
for item in results:
    # 获取每个字典中的"档案类型"并添加到集合中
    archive_types.add(item["档案类型"])

# 输出档案类型数量
print(f"档案类型数量: {len(archive_types)}")

# 输出对应的档案类型，每个类型换行显示
print("对应的档案类型:")
for atype in archive_types:
    print(atype)

档案类型数量: 16
对应的档案类型:
授权委托书

病程记录
参保（农合）住院患者知情同意书
住院期间医患沟通记录
输血（血液制品）治疗知情同意书
手术风险评估表
患者使用高值医用耗材知情选择同意书
出院证
化疗知情同意书
出院记录
<>引产术知情同意书
诊断证明书
入院记录
外周血管及肿瘤介入诊疗知情同意书
心导管诊疗知情同意书


In [9]:
string_result = """放弃积极治疗知情同意书
全脑血管造影术知情同意书
手术知情同意书
室早射频消融手术记录单
溶栓治疗记录
新生儿氧疗知情同意书
产科出院记录
汉密顿抑郁量表（HAMD）
快速病理诊断知情同意书
汉密尔顿焦虑量表HAMA（14项版）
新生儿首次记录单母亲
介入操作检查知情同意书
冠脉造影+支架植入术手术记录单
左侧旁道手术记录单
新生儿黄疸值记录表
知情同意书
消化内镜检查知情同意书
引产术知情同意书
冠脉造影手术记录单
纤维支气管镜检查肺泡灌洗记录
放射治疗知情同意书
拒绝或放弃医学治疗告知书患者
住院期间医患沟通记录
消化内镜检查治疗同意书
婴儿病程
起搏器植入术手术记录单
ASPECTS评分
下肢静脉血栓溶栓治疗签字同意书
参保（城镇居民）住院患者知情同意书
出院证
医患双方不收和不送“红包”协议书
永久起搏器安置术知情同意书
纤维支气管镜检查、治疗及灌洗记录
ACS患者CRUSADE出血风险评分
病危通知书
房颤患者卒中-出血风险评分
重大（特殊）手术审批单申请
引产记录
法定监护人委托书(未成年人或无民事行为能力患者)
中心静脉置管知情同意书
鼓室注药知情同意书
患者使用高值医用耗材知情选择同意书
星状神经节阻滞术操作知情同意书
ERCP手术（操作）知情同意书
转科知情同意书
血浆置换知情同意书患者
死亡病例讨论记录
大隐静脉射频闭合术知情同意书
糖皮质激素治疗知情同意书
HAS-BLED评分
心导管诊疗知情同意书
ABCD2评分量表
非手术患者VTE风险与预防评估表
门诊复诊病历
河南省医疗保健机构胎盘处理告知、处置单
授权委托书
参保（市医保）住院患者知情同意书
化疗知情同意书
手术风险评估表
内镜食管胃底静脉曲张套扎术及硬化剂治疗知情同意书
患者使用高值医用耗材知情选择同意书+
经外周中心静脉置管（PICC）}术
病理诊断知情同意书
入院记录
经皮肝穿刺胆道引流及支架植入术知情同意书
会诊单
病重通知书
放射性粒子植入术知情同意书
病人手术知情同意书
手术记录
连续性肾脏替代治疗知情同意书患者
冠脉介入+冠脉内超声+冠脉内旋磨术手术记录单
房颤射频消融手术记录单
纤支镜引导下经皮微创气管切开记录
GCS评分
营养评估表
颈内静脉透析用长期管路置管知情同意书
诊断证明书
神经介入检查治疗患者或家属知情同意书
介入手术记录
输血（血液制品）治疗知情同意书
重组组织纤溶酶元激活物（rtPA）静脉溶栓治疗知情同意书
内镜手术（操作）知情同意书
参保（农合）住院患者知情同意书
内镜食管扩张术知情同意书
介入操作记录单
简易智力精神状态检查量表MMSE
产科入院医患沟通书
美国国立卫生研究院卒中量表（NIHSS）
mRS评分
肾穿刺造瘘术病人手术知情同意书
静脉血栓栓塞症防治知情同意书
24小时内入出院记录
门诊病历
疼痛治疗知情同意书
冠状动脉介入手术记录单
住院病人告知书
安胎知情同意书
自动出院或转院告知书
心脏电生理介入诊疗知情同意书
全脑血管造影术及支架置入术知情同意书
纤维支气管镜检查及治疗记录
血液透析（滤过）知情同意书患者
肿瘤免疫检査点抑制剂治疗知情同意书
住院患者病情评估表
连续性肾脏替代治疗知情同意书
外周血管及肿瘤介入诊疗知情同意书
住院儿童患者营养不良评估筛查表
入院告知书
GRACE评分表
神经康复评定测定
蒙特利尔认知评估中文版
糖皮质激素应用知情同意书
急诊抢救记录
手术安全记录核查单
院内多学科联合会诊申请表申请会诊
肉毒毒素注射治疗知情同意书
病程记录
CHA2DS2-VASC评分
新生儿危重病例评分表
<欣普贝生>引产术知情同意书
<缩宫素>引产术知情同意书
病危（重）通知书
产科手术同意书
出院记录
住院成人患者营养风险筛查表
参保住院患者知情同意书
纤维支气管镜检查知情同意书患者
药物流产/引产及清宫术知情同意书
临床路径知情同意书
ICU谈话记录
气管插管和机械通气知情同意书患者
中心静脉置管记录
"""

string = """失禁性皮炎评估及措施单
约束护理记录单
伤口护理记录单
日常生活能力评定及措施单
造口护理记录单
护理文件书写质量评分评级表
儿童危重护理记录单（特护）
住院患者营养评估及措施单(NRS2002)
管路滑脱评估及措施单
内镜术前患者转运交接记录单
护理记录单
管路滑脱风险告知书
住院患儿营养评估及措施单（Strong Kids ）
烫伤评估及措施单
吞咽功能障碍初级筛查评估单
约束具使用告知书
出院评估与指导
患者转运交接记录单
烫伤风险告知书
住院患儿跌倒坠床评估及措施单（Humpty Dumpty）
住院患者跌倒坠床评估及措施单(Morse)
输液工具选择风险告知书
深静脉血栓风险告知书
住院患者跌倒坠床告知书
健康教育计划实施记录单
住院患者压力性损伤预警报告表（Braden）
住院患者压力性损伤评估及措施单（Braden）
住院患儿跌倒坠床告知书
住院患儿压力性损伤评估及措施单（BradenQ）
深静脉血栓评估及措施单（Autar）
转入评估单
纤支镜术前患者转运交接记录单
经外周静脉置入中心静脉导管拔管告知书
经外周静脉置入中心静脉导管置入告知书
疼痛评估及措施单
住院患儿压力性损伤预警报告表（BradenQ）
深静脉血栓评估及措施单（Padua）
入院评估单
压力性损伤风险告知书
手术术前患者转运交接记录单
血糖监测单
静脉导管维护记录表
"""

list_a = string_result.strip().split('\n')

list_b = string.strip().split('\n')